In [ ]:
import pennylane as qml
import pennylane.numpy as np
import networkx as nx
from pennylane.transforms import mitigate_with_zne
from matplotlib import pyplot as plt

In [ ]:
graph = nx.Graph(((0, 1), (1, 2), (2, 3), (3,4), (4,5), (0, 5)))
from pennylane.qaoa import maxcut
hamiltonian_maxcut, hamiltonian_mixer = maxcut(graph)
nx.draw(graph, with_labels=True, font_weight='bold')

In [ ]:
obs = []
coeffs = []

for edge in graph.edges():
    coeffs.extend([1.0, 1.0, 1.0])
    obs.extend([qml.PauliX(edge[0]) @ qml.PauliX(edge[1]),
                        qml.PauliY(edge[0]) @ qml.PauliY(edge[1]),
                        qml.PauliZ(edge[0]) @ qml.PauliZ(edge[1])])
H = qml.Hamiltonian(coeffs, obs)
print(H)
num_qubits = len(H.wires)
print("\nVector space dimension: ", num_qubits)

In [ ]:
n_layers = 2
w1 = np.ones((num_qubits), requires_grad=True)
w2 = np.ones((n_layers, num_qubits - 1, 2), requires_grad=True)
def ansatz_cost(w1, w2):
    qml.SimplifiedTwoDesign(w1, w2, wires=range(num_qubits))
    return qml.expval(H)

print(qml.draw(ansatz_cost)(w1,w2))

In [ ]:
dev = qml.device("lightning.qubit", wires=num_qubits)
qn1 = qml.QNode(ansatz_cost,dev)
res1 = qn1(w1, w2)
res1

Let's optimize the square lattice using noise and mitigate using linear extrapolation of the zne method

In [ ]:
noise_gate = qml.DepolarizingChannel #artificial noise
noise_strength = 0.025 #strength of kraus noise

dev_ideal = qml.device("default.mixed", wires=num_qubits) #comparative ideal noiseless device
dev_noisy = qml.transforms.insert(dev_ideal, noise_gate, noise_strength, position="all") #noisy device


scale_factors=[2]

qnode_ideal = qml.QNode(ansatz_cost, dev_ideal) # ideal qnode
qnode_noisy = qml.QNode(ansatz_cost, dev_noisy) #noisy qnode
qnode_mitigated = mitigate_with_zne(qnode_noisy,
    scale_factors=scale_factors,
    folding=qml.transforms.fold_global,
    extrapolate=qml.transforms.poly_extrapolate, extrapolate_kwargs={'order': 2}
) # mitigated qnode

In [ ]:
print("Ideal QNode: ", qnode_ideal(w1, w2))
print("Mitigated QNode: ", qnode_mitigated(w1, w2))
print("Noisy QNode: ", qnode_noisy(w1, w2))

In [ ]:
folded_circuits = [qml.transforms.fold_global(qnode_noisy, scale_factor) for scale_factor in scale_factors]
print(folded_circuits,"\n\n")

for i in range(len(folded_circuits)):
  drawing, ax = qml.draw_mpl(folded_circuits[i])(w1,w2)
  fig = drawing.figure
  """fig.set_figwidth(16)
  fig.set_figheight(2.5)"""
  drawing.show()

In [ ]:
def VQE_run(cost_fn, max_iter, stepsize=0.1):
    """VQE Optimization loop"""
    opt = qml.SPSAOptimizer(maxiter= max_iter)

    # fixed initial guess
    w1 = np.ones((num_qubits), requires_grad=True)
    w2 = np.ones((n_layers, num_qubits - 1, 2), requires_grad=True)

    energy = []

    # Optimization loop
    for n in range(max_iter):
        (w1, w2), prev_energy = opt.step_and_cost(cost_fn, w1, w2)

        energy.append(prev_energy)
        if (n%50 == 0):
          print("Energy optimization at ",n,"th step is : ",energy[-1])

    energy.append(cost_fn(w1, w2)) # final addition to the last updated params
    print("Energy optimization at final step is : ",energy[-1],"\n")
    return energy # energy list


max_iter = 200

energy_ideal = VQE_run(qnode_ideal, max_iter)
energy_noisy = VQE_run(qnode_noisy, max_iter)
energy_mitigated = VQE_run(qnode_mitigated, max_iter)
energy_exact = np.min(np.linalg.eigvalsh(qml.matrix(H)))
print("Exact Ground state: ",energy_exact)

In [ ]:
plt.figure(figsize=(10, 7))

plt.plot(energy_ideal, "-", label="VQE ideal", color = "m",linewidth = 0.9)
plt.plot(np.arange(stop=max_iter,step=1), [energy_exact]*max_iter, "-", label="E_exact", color = "r")
plt.legend(fontsize=10)

plt.xlabel("Iteration", fontsize=10)
plt.ylabel("Energy", fontsize=10)
plt.show()

In [ ]:
print("Exact ground state: ",energy_exact)
print("Ideally optimized after",max_iter," epochs: ",energy_ideal[-1])
print("ideal optimization error:",np.abs((energy_ideal[-1] - energy_exact)/energy_exact))

In [ ]:
print("Error % for 5 qubit ideal opt = ",np.abs((energy_ideal[-1] - energy_exact)/energy_exact)*100,"%")

In [ ]:
energy_noisy = VQE_run(qnode_noisy, max_iter)
energy_mitigated = VQE_run(qnode_mitigated, max_iter)

##### Mitigated results seem wrong

In [ ]:
print("Mitigated result after",max_iter," epochs: ",energy_mitigated[-1])
print("mitigated optimization error:",np.abs((energy_mitigated[-1] - energy_exact)/energy_exact))

In [ ]:
plt.figure(figsize=(10, 7))

plt.plot(energy_ideal, "-", label="VQE ideal", color = "m",linewidth = 0.9)
plt.plot(energy_noisy, "-", label="VQE noisy", color = "k",linewidth = 0.9)
plt.plot(energy_mitigated, "-", label="VQE mitigated", color = "b",linewidth = 0.9)
plt.plot(np.arange(stop=max_iter,step=1), [energy_exact]*max_iter, "-", label="E_exact", color = "r")
plt.legend(fontsize=10)

plt.xlabel("Iteration", fontsize=10)
plt.ylabel("Energy", fontsize=10)
plt.show()